In [ ]:
#!pip install d2l==1.0.0-alpha1.post0 

In [ ]:
from d2l import torch as d2l

# DIY CNN pour les classification

In [ ]:
import os

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pylab as plt
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if device == 'cuda':
    import torch.backends.cudnn as cudnn
    cudnn.benchmark = True


In [ ]:
device

In [ ]:
!nvidia-smi

## Introduction

On va ici créer notre propre réseau de neuronnes

La structure proposé est (conv2 sont de stride 1x1) : 
- **conv1** : 32 2D convolution 3x3 + ReLU activation
- **pool1** : max pooling of 2x2
- **conv2** : 64 convolution of 3x3 + ReLU activation
- **pool2** : max pooling of 2x2 
- **conv3** : 64 convolution of 3x3 + ReLU activation
- **pool3** : max pooling of 2x2
- **fc4** : fully connected with output of 1000 + ReLU activation
- **fc5** : fully connected with output of 10 + SoftMax

On fini sur 10 neuronnes car on va faire de la classification à 10 classes sur CIFAR 10

### Petit conseil :  Essayer de tracer la taille de l'espaces de features et la taille de l'espaces des paramètres au travers de ce réseaux de neuronnes.

## Création du  Dataset

On charge et on ouvre le dataset CIFAR10 (disponnible dans le module torchvision) et on utilise un simple pipeline de transformation qui va seulement charger les images dans un tenseur Pytorch.

In [ ]:
transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True,
                                        transform = transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Un ensemble de données (dataset) est une structure dotée des méthodes `__getitem__` et `__len__` implémentées. Lorsque vous appelez `__getitem__` avec l'opérateur crochet `[x]`, vous accédez au x-ième élément du dataset. Ici, dans le cas de CIFAR-10, cela signifie une image et l'entier associé à l'étiquette de classe.

In [ ]:
I, l = trainset[42]

In [ ]:
I.shape

On remarquera ici que l'image étant un tenseur pytorch, il se situe sur la device de calcul (par exemple le GPU) pour la récupérer on va utiliser les methodes `.cpu().numpy()` pour l'affichage de plus la convention pytorch est `CxHxW` (Channel puis hauteur puis largeurs)  alors que la convention d'affichage de matplotlib est `HxWxC` on utilise donc une permutation pour intervertir la première dimension et la dèrniere :

In [ ]:
plt.figure()
plt.imshow(I.cpu().numpy().transpose([1,2,0]))

In [ ]:
I.cpu().numpy().transpose([1,2,0]).shape

In [ ]:
classes[l]

Affichons quelques illustrations

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    _ = plt.axis('off')

In [ ]:
plt.figure(figsize=(9,8))
for x in range(4):
    for y in range(4):
        idx = x + 4*y
        I, L = trainset[idx]
        plt.subplot(4,4,idx+1)
        imshow(I)
        plt.title(classes[L])

## Network creation

Un réseau de neurones est simplement une classe qui hérite du module torch.nn et qui implémente la propagation avant (forward pass), laquelle calcule la sortie du réseau neuronal sur un mini-batch :

Par exemple je vous fournis l'implémentation du réseau suivant : 
- **conv1** 6 channels output, 5x5 | ReLU
- **pool** 2x2 max pooling
- **conv2** 16 channels output, 5x5 | ReLU
- **pool** 2x2 max pooling
- **fc1** 120 ouputs | ReLU
- **fc2** 84 outputs | ReLU
- **fc3** 10 outputs | ReLU

In [ ]:
class SampleNet(nn.Module):
    def __init__(self):
        super(SampleNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        self.activation = F.relu

    def forward(self, x):
        # if we decompose we have
        x = self.conv1(x)
        x = self.activation(x)
        x = self.pool(x)
        # in one line we have 
        x = self.pool(self.activation(self.conv2(x)))
        
        # here we reshape the output in [batch_size, 16x5x5] Tensor :
        x = x.view(-1, 16 * 5 * 5)
        # then we apply the fully connected
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        return x

In [ ]:
sample = SampleNet()

Si on veut voir la structure du réseau on peut simplement l'afficher : 

In [ ]:
print(sample)

Une solution pour déboguer le réseau de neurones ou pour découvrir l'entrée de notre réseau est de l'éprouver avec `torch.zeros(Bs, C, H, W)` comme entrée et de chercher les bonnes valeurs de C, H, W, par exemple dans la cellule suivante j'ai trouver les dimensions d'entrée de SampleNet : 

In [ ]:
H0 = 32
C0 = 3
W0 = 32
sample.forward(torch.zeros(16, C0, H0, W0)).shape


## C'est a vous de jouer et d'implmémenter ClassiNet : 

- **conv1** : 32 2D convolution 3x3 then ReLU activation
- **pool1** : max pooling of 2x2
- **conv2** : 64 convolution of 3x3 then ReLU activation
- **pool2** : max pooling of 2x2 
- **conv3** : 64 convolution of 3x3 then ReLU activation
- **pool3** : max pooling of 2x2
- **fc4** : fully connected with output of 1000 then ReLU activation
- **fc5** : fully connected with output of 10 then SoftMax

In [ ]:
class ClassiNet(nn.Module):
    def __init__(self):
        super(ClassiNet, self).__init__()
        # define your layers here

        self.activation = ...

    def forward(self, x):
        # make your forward here 
        return x

La cellule suivante sert à valider votre implémentation :

In [ ]:
H0 = 32
C0=3
W0=32
classi = ClassiNet()
classi.forward(torch.zeros(16, C0, H0, W0)).shape

On peut simplement calculer le nombre de paramètres de notre réseau de neuronnes en sommant le nombre de parametres par couches :

In [ ]:
np.sum([x.numel() for x in list(classi.parameters())])

## Trainning fonction   


Pour simplifier le codage on va définir deux fonction 

- train_batch qui est la phase se forward > calcul de la loss > retropropagation d'un mini-batch

- train qui est la boucle total de l'apprentissage (le `fit`) et qui nous fera aussi un beau plot avec le suivit de la loss et de l'accuracy en test et train

In [ ]:
def train_batch(net, X, y, loss, trainer, devices):
    """Train for a minibatch with multiple GPUs capabilities"""
    # on met notre batch de donnée sur la cible de calcul
    if isinstance(X, list):
        X = [x.to(devices[0]) for x in X]
    else:
        X = X.to(devices[0])
    y = y.to(devices[0])
    # on met notre reseau en mode 'train'
    net.train()
    # on initialise le gradient a zéro
    trainer.zero_grad()
    # on fait la prediction
    pred = net(X)
    # on calcul la lfonction de perte 
    l = loss(pred, y)
    # on fait la rétro-propagation du gradient
    l.sum().backward()
    # on informe la descente de gradient qu'on veut faire un pas dans la descente
    trainer.step()
    # on sauvegarde les informations sur les prédiction en train
    train_loss_sum = l.sum()
    train_acc_sum = d2l.accuracy(pred, y)
    return train_loss_sum, train_acc_sum

On voit ici que la grande modularité de Pytorch nous permettra de changer l'optimisateur (`trainer`) et de choisir plus tard Adam / SGD ou tout autre optimiseurs.

Regardons un peut plus en détail la boucle d'apprentissage :

In [ ]:
def train(net, train_iter, test_iter, loss, trainer, num_epochs,
               devices=d2l.try_all_gpus()):
    """Trainning loop
    
    net : le reseau de neuronnes 
    train_iter : le dataloder train
    test_iter : le dataloader test
    loss : la fonction de cout choisi
    trainner : l'optimiseur choisi
    num_epoch : le nombre d'époch 
     """
    timer, num_batches = d2l.Timer(), len(train_iter)

    # plot annimation
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0, 1],
                            legend=['train loss', 'train acc', 'test acc'])
    
    # declare network for multiple GPU :
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])

    for epoch in range(num_epochs):
        # Sum of training loss, sum of training accuracy, no. of examples,
        # no. of predictions
        metric = d2l.Accumulator(4)
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            
            # do one batch : 
            l, acc = train_batch(
                net, features, labels, loss, trainer, devices)
            
            metric.add(l, acc, labels.shape[0], labels.numel())
            timer.stop()
            
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (metric[0] / metric[2], metric[1] / metric[3],
                              None))
        
        test_acc = d2l.evaluate_accuracy_gpu(net, test_iter)

        animator.add(epoch + 1, (None, None, test_acc))
    print(f'loss {metric[0] / metric[2]:.3f}, train acc '
          f'{metric[1] / metric[3]:.3f}, test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec on '
          f'{str(devices)}')

## L'apprentissage 

Pour l'apprentissage nous avons fait les choix suivant :  

- une loss function de type CrossEntropy
- l'optimiseur SGD (descente stockastique)


Les principaux hyper-paramètres sont : 

- le nombe d'epoch (nombre de fois ou on itère sur le dataset)
- le learning_rate : le pas de descente
- momentum :  lissage de la direction du gradient
- batch_size : le nombre d'élément dans un mini-batch

In [ ]:
epoch_number = 8
learning_rate = 0.01
momentum = 0.6 
batch_size = 16

On construit un dataloader qui est une classe Pytorch qui encapsule le dataset et en particulier permet des chargements parallèles des données : 

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=8)

On consturit un autre dataloader mais sur l'ensemble de test cette fois-ci (qui sera un ensemble de donnée disjoint de l'ensemble de train)

In [ ]:
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=8)

Notre fonction de cout sera une Cross Entropie, un choix classique pour de la classification : 

In [ ]:
loss_crit = nn.CrossEntropyLoss()

On construit notre réseau de neuronnes et on le déplace sur le device de calcul. 

Et on choisi un optimiseur : 

In [ ]:
classi = ClassiNet().to(device)

optimizer = optim.SGD(classi.parameters(), lr=learning_rate, momentum=momentum)

Trainning loop (~10 min on GPU) :

In [ ]:
train(classi, trainloader, testloader, loss_crit, optimizer, epoch_number)

## Question] qu'observez-vous sur ce plot? est-il utile d'itérer plus?

essayer différents hyper paramètres tel que le learning rate / batch_size voir changer l'optimizeur pour voir si vous pouvez améliorer les performances 

## Calcul de la matrice de confusion 

Afin d'obtenir une vision plus précise des performances de notre réseau, il est judicieux de visualiser la matrice de confusion. Pour ce faire, nous commençons par écrire une petite fonction pour évaluer les données sur l'ensemble de test :



In [ ]:
def computePerf(test, net):
    """
        From the test dataloader and the trained network compute the y_pred and y_true
        vector
    """
    y_pred = []
    y_true = []
    with torch.no_grad():
        for data in test:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            y_pred.extend(predicted.cpu().numpy()) # Save Prediction
            y_true.extend(labels.cpu().numpy()) # Save Truth
    return y_pred, y_true


In [ ]:
y_pred, y_true  = computePerf(testloader, classi)

Calcul de la matrice de confusion  :

In [ ]:
cf_matrix = confusion_matrix(y_true, y_pred)

Affichage de la matrice de confusion :

In [ ]:
df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) *10, index = [i for i in classes],
                     columns = [i for i in classes])
plt.figure(figsize = (12,7))
sn.heatmap(df_cm, annot=True)

## Question ] Analysez cette matrice :

- quels sont les classes les mieux classé?
- quels sont les classes que le réseau confond le plus?

La précision moyenne peut simplement s'obtennir en calculant la some de la trace de la matrice de confusion :







In [ ]:
def getAccuracy(conf_mat):
    return np.trace(conf_mat/np.sum(conf_mat))

## Plus réaliste : 

En général on aura tendance a repartir de models pré-éxistant, par exemple dans la cellule suivante on prend un vgg16, cependant dans le cadre de la classification il faudra toujours changer le nombre de couches final et le remplacer par une couche linéaire de bonne taille en sortie (10 classes pour CIFAR 10) :

In [ ]:
from torchvision import models

def getNetwork():
    model = models.vgg16(pretrained = True)
    input_lastLayer = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(input_lastLayer,10)
    model = model.to(device)
    return model
classi = getNetwork()

# Augmentation de données 

Les grand jeux de données sont un pré-requis à l'apprentissage profond. Lorsqu'on ne dispose pas d'un ensemble suffisant de donnée une solution est de faire de l'augmentation de donnée qui appliquera des transformation alléatoire (rotation, éclairage, translation, bruit, ... ) pour augmenter artificiellement la taille de notre dataset.

C'est ce que nous allons mettre en oeuvre ici 


## Quelques exemples d'augmentation d'images 

Dans cette section nous allons faire quelques tests sur un chat : 

In [ ]:
d2l.set_figsize()
!wget -nv https://upload.wikimedia.org/wikipedia/commons/thumb/5/5d/Bengal_cat1.jpg/450px-Bengal_cat1.jpg -O cat1.jpg
img = d2l.Image.open('./cat1.jpg')
d2l.plt.imshow(img);

La plupart des méthodes d'augmentation de données possèdent un certain degré d'aléatoire. Afin de nous faciliter l'observation de l'effet de l'augmentation d'image, nous définissons ensuite une fonction auxiliaire `apply`. Cette fonction exécute la méthode d'augmentation d'image `aug` plusieurs fois sur l'image d'entrée `img` et montre tous les résultats.


In [ ]:
def apply(img, aug, num_rows=2, num_cols=4, scale=1.5):
    Y = [aug(img) for _ in range(num_rows * num_cols)]
    d2l.show_images(Y, num_rows, num_cols, scale=scale)

### Flipping and Cropping


[**Flipping the image left and right**] ne modifie généralement pas la catégorie de l'objet. Cela constitue l'une des premières méthodes et l'une des plus largement utilisées de l'augmentation d'image. Ensuite, nous utilisons le module `transforms` pour créer l'instance `RandomHorizontalFlip`, qui inverse une image de gauche à droite avec une probabilité de 50%.

In [ ]:
apply(img, torchvision.transforms.RandomHorizontalFlip())

[**Flipping up and down**] 


In [ ]:
apply(img, torchvision.transforms.RandomVerticalFlip())

Dans l'image d'exemple que nous avons utilisée, le chat se trouve au milieu de l'image, mais cela ne peut pas être le cas en général. 

Dans le code ci-dessous, nous **crop aimons aléatoirement** une zone dont l'aire est comprise entre $10\% \sim 100\%$ de l'aire originale à chaque fois, et le rapport de la largeur à la hauteur de cette zone est sélectionné aléatoirement parmi les valeurs de $0.5 \sim 2$. Ensuite, la largeur et la hauteur de la région sont toutes les deux mises à l'échelle pour atteindre 200 pixels.

Sauf indication contraire, le nombre aléatoire compris entre $a$ et $b$ dans cette section fait référence à une valeur continue obtenue par échantillonnage aléatoire et uniforme de l'intervalle $[a, b]$.

In [ ]:
shape_aug = torchvision.transforms.RandomResizedCrop(
    (200, 200), scale=(0.1, 1), ratio=(0.5, 2))
apply(img, shape_aug)

### Modification de la colorimétrie

On peut ajouter un peut de bruit sur la couleur en modifiant illumination / contrast et staturation : 

In [ ]:
apply(img, torchvision.transforms.ColorJitter(
    brightness=0.5, contrast=0.1, saturation=0, hue=0))

Plus esthétique on peut aussi changer la composante de couleure :



In [ ]:
apply(img, torchvision.transforms.ColorJitter(
    brightness=0, contrast=0, saturation=0, hue=0.5))

On peut aussi tout faire a la fois : 

In [ ]:
color_aug = torchvision.transforms.ColorJitter(
    brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5)
apply(img, color_aug)

### Combining Multiple Image Augmentation Methods
En pratique, nous **associerons plusieurs méthodes d'augmentation d'image**. Par exemple, nous pouvons combiner les différentes méthodes d'augmentation d'image présentées précédemment et les appliquer à chaque image via une instance de `Compose`.

In [ ]:
augs = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(), color_aug, shape_aug])
apply(img, augs)

## Apprentissage avec Augmentation

Reprenons les apprentissages de CIFAR 10 mais avec une augmentation de donnée cette fois-ci


In [ ]:
d2l.show_images([trainset[i][0].cpu().numpy().transpose(1,2,0) for i in range(32)], 4, 8, scale=0.8);

Il est important de dnoter que les augmentation de données ont lieu uniquement sur les jeux de données d'apprentissage, il n'y a pas de sens à les utiliser sur l'ensemble de test (sauf pour des raison de débug)

In [ ]:
train_augs = torchvision.transforms.Compose([
     torchvision.transforms.RandomHorizontalFlip(),
     torchvision.transforms.ToTensor()])

test_augs = torchvision.transforms.Compose([
     torchvision.transforms.ToTensor()])

Une fonction de simplification qui construit le dataloader sur cifar10

In [ ]:
def load_cifar10(is_train, augs, batch_size):
    dataset = torchvision.datasets.CIFAR10(root="../data", train=is_train,
                                           transform=augs, download=True)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                    shuffle=is_train, num_workers=d2l.get_dataloader_workers())
    return dataloader

On va apprendre un resnet18 histoire d'utiliser un réseau de neuronne de l'état de l'art.


In [ ]:
net = d2l.resnet18(10, 3)
net.apply(d2l.init_cnn)

batch_size  = 256

In [ ]:
def train_with_data_aug(train_augs, test_augs, net, lr=0.001):
    train_iter = load_cifar10(True, train_augs, batch_size)
    test_iter = load_cifar10(False, test_augs, batch_size)
    loss = nn.CrossEntropyLoss(reduction="none")
    trainer = torch.optim.Adam(net.parameters(), lr=lr)
    net(next(iter(train_iter))[0])
    train(net, train_iter, test_iter, loss, trainer, 10)

REgardons ce que donne l'apprentissage avec augmentations : 




In [ ]:
train_with_data_aug(train_augs, test_augs, net)

In [ ]:
train_with_data_aug(test_augs, test_augs, net)

## Exercises

1. tester l'impact de l'augmentation de donné sur l'apprentissage sur Cifar10
1. jouer sur les méta-parametres ( learning rate / batch size / ...) 
1. Normalement vous devrirez pouvoir atteindre de meilleur performances en test accuracy grâce a l'augmentation de donnée en limittant l'effet d'overfiting (sur-apprentissage), essayer de voir jusqu'ou vous pouvez aller.

# Finetunage

Dans cette section nous allons mettre en pratique le fine-tunage, c'est a dire que nous allons récupérer un réseau de neuronne appris sur image-net et chercher à principallement réaprendre que la dèrnier couche.

Pour cela dans pytorch cela va correspondre à fournir à l'optimizer un learning rate variable en fonction des couches (faible pour les couches a ne pas ré-apprendre et fort pour les couches à ré-apprendre).


## Hot Dog Recognition

De tout temps l'homme a chercher à reconnaitre des hot-dog, c'est pour cela que nous nous attacherons à cette tâche 


### Reading the Dataset

On a récupérer d'internet 1400 images de hot-dog et le même nombre d'images d'autre choses. On utilisera 1000 images par classes pour le train et 400 pour le test. 

Le code suivant télécharge et dezip les données : 

In [ ]:
d2l.DATA_HUB['hotdog'] = (d2l.DATA_URL + 'hotdog.zip',
                         'fba480ffa8aa7e0febbb511d181409f899b9baa5')

data_dir = d2l.download_extract('hotdog')

On créer nos classes datasets :

In [ ]:
train_imgs = torchvision.datasets.ImageFolder(os.path.join(data_dir, 'train'))
test_imgs = torchvision.datasets.ImageFolder(os.path.join(data_dir, 'test'))

The first 8 positive examples and the last 8 negative images are shown below. As you can see, the images vary in size and aspect ratio.


In [ ]:
hotdogs = [train_imgs[i][0] for i in range(8)]
not_hotdogs = [train_imgs[-i - 1][0] for i in range(8)]
d2l.show_images(hotdogs + not_hotdogs, 2, 8, scale=1.4);

## Data augmentations

Au cours de l'entraînement, nous coupons d'abord une zone aléatoire de taille et de rapport d'aspect aléatoires dans l'image, puis mettons à l'échelle cette zone pour obtenir une image d'entrée de $224 \times 224$. Lors des tests, nous mettons à l'échelle la hauteur et la largeur d'une image à 256 pixels, puis coupons une zone centrale de $224 \times 224$ comme entrée. De plus, pour les trois canaux de couleur RVB (rouge, vert et bleu), nous *standardisons* leurs valeurs canal par canal. Concrètement, la valeur moyenne d'un canal est soustraite de chaque valeur de ce canal, puis le résultat est divisé par l'écart type de ce canal.

In [ ]:
# Specify the means and standard deviations of the three RGB channels to
# standardize each channel
normalize = torchvision.transforms.Normalize(
    [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_augs = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    normalize])

test_augs = torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    normalize])

## Création du model 

On va utiliser un resnet18 pour cet exercice, il sera pré-appris sur Imagenet

In [ ]:
pretrained_net = torchvision.models.resnet18(pretrained=True)

Il est utile de connaitre le nombre de features de sortie du réseau, pour cela il suffit d'afficher la dèrniere couche (`fc` sur un resnet18) : 

In [ ]:
pretrained_net.fc

Question : Quel est le nombre de features de sortie du réseau et pourquoi?

Question : dans notre cas combien de features de sortie il nous faudra? et pourquoi ?

Création d'un réseau fine_tuné et remplacage de la couche de sortie (et son initialisation ) :

In [ ]:
finetune_net = torchvision.models.resnet18(pretrained=True)
finetune_net.fc = nn.Linear(finetune_net.fc.in_features, 2)
nn.init.xavier_uniform_(finetune_net.fc.weight);

## Fine-Tunage du Model




Pour finetuner uniquement une certaine couche il suffit de ne donner que les paramètres qu'il faut optimiser a l'Optimiseur par exemple ligne 20 de la cellule suivante on va définir un learning rate 10x plus grand sur la couche de sortie que sur le reste du réseau. Ainsi l'erreur sera principallement rétropropagé sur cette partie du réseau.

In [ ]:

def train_fine_tuning(net, learning_rate, batch_size=128, num_epochs=5,
                      param_group=True):
    # construct the Dataloader :
    train_iter = torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(
        os.path.join(data_dir, 'train'), transform=train_augs),
        batch_size=batch_size, shuffle=True)
    test_iter = torch.utils.data.DataLoader(torchvision.datasets.ImageFolder(
        os.path.join(data_dir, 'test'), transform=test_augs),
        batch_size=batch_size)
    
    devices = d2l.try_all_gpus()
    # The Loss : 
    loss = nn.CrossEntropyLoss(reduction="none")

    # The optimizer :     
    if param_group:
        params_1x = [param for name, param in net.named_parameters()
             if name not in ["fc.weight", "fc.bias"]]
        trainer = torch.optim.SGD([{'params': params_1x},
                                   {'params': net.fc.parameters(),
                                    'lr': learning_rate * 10}
                                   ],
                                lr=learning_rate, weight_decay=0.001)
    else:
        trainer = torch.optim.SGD(net.parameters(), lr=learning_rate,
                                  weight_decay=0.001)
    d2l.train_ch13(net, train_iter, test_iter, loss, trainer, num_epochs,
                   devices)

On lance l'apprentissage : 

In [ ]:
train_fine_tuning(finetune_net, 5e-5)

## Apprentissage from-scratch

pour comparaison on peut faire le même exercice mais en apprenant le resnet complet non pre-trainé

In [ ]:
scratch_net = torchvision.models.resnet18()
scratch_net.fc = nn.Linear(scratch_net.fc.in_features, 2)
train_fine_tuning(scratch_net, 5e-4, param_group=False)

## Exercices : 

1. experimentez différent type de fine-tunage (jouez sur le ratio du learning rate entre les deux partie du réseau)
1. tester sur des réseaux de neuronnes plus gros par exemple `torchvision.models.resnet101()`